In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
# from mplfinance import candlestick_ohlc
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np

from os.path import exists

from datetime import datetime
import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6

from dotmap import DotMap

import seaborn as sns
# Apply the default theme
sns.set_theme()


matplotlib.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})


In [254]:
def ffill_missing(df, tframe):
  freq = pd.Timedelta(tframe)
  td_since_last = df.date.diff()
  missing = td_since_last[td_since_last != freq].tail(-1) .to_dict()
  df = pd.concat([df, *[pd.DataFrame(index=pd.date_range(start=k-v+freq, end=k-freq, freq=freq)) for k,v in missing.items()]])
  df.sort_index(inplace=True)
  df.ffill(inplace=True)
  return df

def load_json_candles(fname):
  df = pd.read_json(fname)
  df.columns = ['ts', 'o', 'h', 'l', 'c', 'v']

  df['date'] = pd.to_datetime(df['ts'], unit='ms', utc=False)
  df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
  df.sort_index(inplace=True)
  return df


In [258]:
symbol = 'ALGO_USDT'
tframe_H = '4h'
tframe_L = '15m'
data_folder = './freq-user-data/data/binance/'
df_H = load_json_candles(f'{data_folder}/{symbol}-{tframe_H}.json')
df_L = load_json_candles(f'{data_folder}/{symbol}-{tframe_L}.json')
df_H = ffill_missing(df_H, tframe_H)
df_L = ffill_missing(df_L, tframe_L)

In [259]:

# df.ffill(inplace=True)
# df_L.loc['2021-03-06 01:30:00': '2021-03-06 03:30:00', ['date', 'o', 'h', 'l', 'c']]

In [ ]:
def add_atr(df, periods):
  high_low = df['h'] - df['l']
  high_close = np.abs(df['h'] - df['c'].shift())
  low_close = np.abs(df['l'] - df['c'].shift())
  ranges = pd.concat([high_low, high_close, low_close], axis=1)
  true_range = np.max(ranges, axis=1)
  atr = true_range.rolling(periods).sum()/periods
  df['tr'] = true_range
  df['atr'] = atr
  df['tr_N'] = true_range / df.c
  df['atr_N'] = atr / df.c
  return df

def add_stoch(df, k_period, d_period):
  # Adds a "n_high" column with max value of previous 14 periods
  df['n_high'] = df['h'].rolling(k_period).max()
  # Adds an "n_low" column with min value of previous 14 periods
  df['n_low'] = df['l'].rolling(k_period).min()
  # Uses the min/max values to calculate the %k (as a percentage)
  df['stoch_k'] = (df['c'] - df['n_low']) * 100 / (df['n_high'] - df['n_low'])
  # Uses the %k to calculates a SMA over the past 3 values of %k
  df['stoch_d'] = df['stoch_k'].rolling(d_period).mean()

In [ ]:
def add_logs(df, columns):
  for column in columns:
    df[f'{column}_log'] = df[column].apply(np.log)

In [ ]:
# df_H.iloc[0].date
# df_L.loc[wdf_H.iloc[0].date:wdf_H.iloc[-1].date]
# len(wdf_H), len(wdf_L)
# add_logs(df_H, ['o', 'c'])
# df_H

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1, 1)
ax.plot(df_H.head(100).c)

In [ ]:
# dir(ax.yaxis)
# ax.yaxis.get_view_interval()
# ax.axvspan
# g = wdf_H.groupby('g_no')
# dir(g)
[(v[0], v[-1]) for  v in g.groups.values()]
# wdf_H

In [169]:

def add_groups(df, cond_in, cond_out, prefix='g_'):
    cond_start = ( cond_in & ~cond_in.shift(fill_value=False) )
    # cond_end = ( cond_out & ~cond_out.shift(fill_value=False) )

    start_nos = cond_start.cumsum()
    cond_end = ( (start_nos > 0) & cond_out & ~cond_out.shift(fill_value=False) )
    end_nos = cond_end.cumsum()

    is_in = (start_nos > end_nos)

    df[f'{prefix}no'] = start_nos
    df[f'{prefix}start'] = cond_start
    df[f'{prefix}end'] = cond_end
    # df['g_cond_beg_end'] = (cond_beg | cond_end)
    df[f'{prefix}in'] = is_in 
    return df

def copy_data_from_H(df_L, columns, df_H):
    for column in columns:
        col_name = f'{column}_H'
        df_L.loc[df_H.index, col_name] = df_H[column]
        df_L[col_name] = df_L[col_name].ffill() 

In [266]:
strategy = 'i-2-timeframes'
strategy_params_json = f'./strategy-params/{strategy}.json'

dlen = len(df_H)
sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

strategy_params = {
    'atr_H': {'type': 'int', 'value': 14, 'step': 1},
    'ma1_H': {'type': 'int', 'value': 14, 'step': 1},
    'ma2_H': {'type': 'int', 'value': 26, 'step': 1},
    'ma3_H': {'type': 'int', 'value': 50, 'step': 1},

    'ma1_L': {'type': 'int', 'value': 14, 'step': 1},
    'ma2_L': {'type': 'int', 'value': 26, 'step': 1},
    'ma3_L': {'type': 'int', 'value': 50, 'step': 1},
    'st_L_k': {'type': 'int', 'value': 14, 'step': 1},
    'atr_L': {'type': 'int', 'value': 14, 'step': 1},
    # 'mav1mav2': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    # 'mav2mav3': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    # 'stop_gap': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    # 'slipage': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    # 'risk': {'type': 'float', 'value': 0.01, 'step': 0.005},
    # 'sl': {'type': 'float', 'value': 0.01, 'step': 0.001},
    # 'r2r': {'type': 'float', 'value': 2./1., 'step': 0.05},
    # 'wbtp': {'type': 'float', 'value': 0.1, 'step': 0.05},
    # 'trail': {'type': 'float', 'value': 0.02, 'step': 0.005},
}
for pk, pv in strategy_params.items():
    pv['wdg'] = globals()[(pv['wdg'] if 'wdg' in pv else pv['type'].capitalize() + 'Text')](
        description=pk,disabled=False,continuous_update=False,
        value=pv['value'], step=pv['step']
    )
strategy_params = {"w2log": {'wdg': sl_n2}, "w": {'wdg': sl_w}, **strategy_params}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

wst = wed = wsz = False
wdf_H = wdf_L = False

def printer(w2log, w, **kwargs):
    params = DotMap(kwargs)
    # global capital
    capital = 10000
    global wst, wed, wsz, wdf_H, wdf_L
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))

    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    ##################################################
    ################ Higher Timeframe ################

    wdf_H = df_H.iloc[wst:wed,:].copy()
    wdf_H['lret'] = wdf_H.c.apply(np.log).diff()
    
    ### Indicators
    wdf_H['ma1'] = wdf_H.c.ewm(span=params.ma1_H, adjust=False).mean()
    wdf_H['ma2'] = wdf_H.c.ewm(span=params.ma2_H, adjust=False).mean()
    wdf_H['ma3'] = wdf_H.c.ewm(span=params.ma3_H, adjust=False).mean()

    add_atr(wdf_H, params.atr_H)

    ### Filter groups

    cond_in_H = (
        (wdf_H.ma1 > wdf_H.ma2) & 
        (wdf_H.ma2 > wdf_H.ma3)
    )
    cond_out_H = (~cond_in_H)
    

    add_groups(wdf_H, cond_in_H, cond_out_H, prefix='maH_')
    maH_inside = wdf_H[wdf_H.maH_in]
    maH_groups = maH_inside.groupby(['maH_no'])


    #################################################
    ################ Lower Timeframe ################

    wdf_L = df_L.loc[wdf_H.iloc[0].date:wdf_H.iloc[-1].date].copy()
    wdf_L['lret'] = wdf_L.c.apply(np.log).diff()
    # wlen = len(wdf)

    ### Indicators
    wdf_L['ma1'] = wdf_L.c.ewm(span=params.ma1_L, adjust=False).mean()
    wdf_L['ma2'] = wdf_L.c.ewm(span=params.ma2_L, adjust=False).mean()
    wdf_L['ma3'] = wdf_L.c.ewm(span=params.ma3_L, adjust=False).mean()

    copy_data_from_H(wdf_L, ['ma1', 'ma2', 'ma3', 'tr_N', 'atr_N', 'maH_no', 'maH_start', 'maH_end', 'maH_in'], wdf_H)
    # wdf_L.loc[wdf_H.index, 'ma1_H'] = wdf_H['ma1']
    # wdf_L.loc[wdf_H.index, 'ma2_H'] = wdf_H['ma2']
    # wdf_L.loc[wdf_H.index, 'ma3_H'] = wdf_H['ma3']
    # wdf_L.ma1_H = wdf_L.ma1_H.ffill()
    # wdf_L.ma2_H = wdf_L.ma2_H.ffill()
    # wdf_L.ma3_H = wdf_L.ma3_H.ffill()

    add_stoch(wdf_L, params.st_L_k, 3)
    add_atr(wdf_L, params.atr_L)
    
    plt.close("all")
    # vlines=list(wdf[wdf['g_cond_beg_end']].index.astype('string').values)

    with plt.rc_context():
        pkwargs = dict(warn_too_much_data=100000)

        wdf_H = wdf_H.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})        
        wdf_L = wdf_L.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})
        fig = plt.figure(constrained_layout=True,figsize=(16,5))
        gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

        gs00 = gs0[0,0].subgridspec(5, 1, height_ratios=[2,1,3,1,1])

        ax11 = fig.add_subplot(gs00[0,0])
        ax12 = fig.add_subplot(gs00[1,0], sharex=ax11)
        # ax11.tick_params(direction='out', axis='y')

        ax21 = fig.add_subplot(gs00[2,0])
        ax22 = fig.add_subplot(gs00[3,0], sharex=ax21)
        ax23 = fig.add_subplot(gs00[4,0], sharex=ax21)


        locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
        formatter = mdates.ConciseDateFormatter(locator)

        ax11.xaxis.set_major_locator(plt.NullLocator())
        ax12.xaxis.set_major_locator(plt.NullLocator())
        # ax21.xaxis.set_major_locator(plt.NullLocator())
        ax21.xaxis.set_major_locator(plt.NullLocator())
        ax22.xaxis.set_major_locator(plt.NullLocator())
        ax23.xaxis.set_major_locator(plt.NullLocator())
        # ax23.xaxis.set_major_formatter(formatter)

        # gs01 = gs0[0,1].subgridspec(4, 1)
        # ax101 = fig.add_subplot(gs01[0,0])
        # ax102 = fig.add_subplot(gs01[1,0])
        # ax103 = fig.add_subplot(gs01[2,0])
        
        # ax11.xaxis.set_major_locator(locator)
        # ax11.xaxis.set_major_formatter(formatter)

######  Higher Timeframe Plots

        # vlines=list(wdf[wdf['g_cond_beg_end']].index.astype('string').values)
        # vlines_H = wdf_H[wdf_H.g_start | wdf_H.g_end].index.values
        
        groups_H = [(v[0], v[-1]) for  v in maH_groups.groups.values()]

        mpf.plot(wdf_H,
            **{**pkwargs, **dict(volume=False, style='binance',type='candle')},  #figsize=(15, 5)
            show_nontrading=True,
            # vlines=dict(vlines=vlines, linewidths=(1,0.5), colors=('g','c'), linestyle='-.'),
            ax=ax11,
            )

        # ymin_H = wdf_H.low.min()
        # ymax_H = wdf_H.high.max()
        ymin_11,ymax_11 = ax11.yaxis.get_view_interval()
        # ax11.vlines(vlines_H, ymin=ymin_11, ymax=ymax_11)

        ax11.plot(wdf_H.ma1, color='orange')
        ax11.plot(wdf_H.ma2, color='green')
        ax11.plot(wdf_H.ma3, color='blue')
        ax12.plot(wdf_H.atr_N, color='b')
        ymin_12,ymax_12 = ax12.yaxis.get_view_interval()
        # ax12.vlines(vlines_H, ymin=ymin_12, ymax=ymax_12)
        for (xmin, xmax) in groups_H:
            ax11.axvspan(xmin, xmax, alpha=0.2)
            ax12.axvspan(xmin, xmax, alpha=0.2)
        # ax12.plot(wdf_H.tr_N, color='orange')

######  Lower Timeframe Plots
        if len(wdf_L) < 10000 :
            mpf.plot(wdf_L,
                **{**pkwargs, **dict(volume=False, style='binance',type='candle')},  #figsize=(15, 5)
                show_nontrading=True,
                # vlines=dict(vlines=vlines, linewidths=(1,0.5), colors=('g','c'), linestyle='-.'),
                ax=ax21,
                )
            
            ax21.plot(wdf_L.ma1, color='orange')
            ax21.plot(wdf_L.ma2, color='green')

        ax21.plot(wdf_L.ma3, color='blue')

        # ax21.plot(wdf_H.ma1, color='orange', linestyle='dashed', lw=0.5)
        # ax21.plot(wdf_H.ma2, color='green', linestyle='dashed', lw=0.5)
        # ax21.plot(wdf_H.ma3, color='blue', linestyle='dashed', lw=0.5)

        ax21.plot(wdf_L.ma1_H, color='orange', linestyle='dotted', lw=0.5)
        ax21.plot(wdf_L.ma2_H, color='green', linestyle='dotted', lw=0.5)
        ax21.plot(wdf_L.ma3_H, color='blue', linestyle='dotted', lw=0.5)

        for (xmin, xmax) in groups_H:
            ax21.axvspan(xmin, xmax, alpha=0.2)
            ax22.axvspan(xmin, xmax, alpha=0.2)

        ax22.plot(wdf_L.stoch_k, color='orange', linestyle='dashed')        
        ax22.plot(wdf_L.stoch_d, color='blue')     
        ax22.axhline(20, color='gray', linestyle='dashed')
        ax22.axhline(80, color='gray', linestyle='dashed')
    
        ax23.plot(wdf_L.atr_N, color='b')
        # ax23.plot(wdf_L.atr_N_H, color='gray', linestyle='dashed')
        # max_in_lret = wdf[['inlret', 'r_inlret']].max().max()
        # min_in_lret = wdf[['inlret', 'r_inlret']].min().min()
        # ax002.plot(wdf.r_inlret, lw=1, color='r')
        # ax002.plot(wdf.max_r_inlret, lw=1, color='g', linestyle='dashed')
        # ax002.plot(wdf.min_r_inlret, lw=1, color='r', linestyle='dashed')
        # ax002.vlines(wdf[wdf.g_cond_beg_end].index, min_in_lret, max_in_lret, lw=0.5, color="c")

        # ax003.plot(wdf.fair_c_ma2, lw=1, color='g')
       
    plt.show()
    
if exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];
                                        

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()})
display(ui, out)

Output()

In [262]:

len(wdf_L)

32753

In [221]:
missing

{Timestamp('2021-03-06 03:30:00'): Timedelta('0 days 01:45:00')}

,ts,open,high,low,close,v,date,lret,ma1,ma2,...,n_high,n_low,stoch_k,stoch_d,tr,atr,tr_N,atr_N,td_since_last,missing
2021-03-06 01:30:00,1.614994e+12,1.0454,1.0491,1.0442,1.0473,152448.35,2021-03-06 01:30:00,0.001911,1.059112,1.062353,...,1.0796,1.0438,9.776536,9.041431,0.0049,0.011171,0.004679,0.010667,0 days 00:15:00,NaT
2021-03-06 01:45:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 02:00:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 02:15:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 02:30:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 02:45:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 03:00:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 03:15:00,1.614995e+12,1.0470,1.0527,1.0470,1.0501,162719.02,2021-03-06 01:45:00,0.002670,1.057911,1.061445,...,1.0776,1.0438,18.639053,10.556462,0.0057,0.011250,0.005428,0.010713,0 days 00:15:00,NaT
2021-03-06 03:30:00,1.615001e+12,1.0498,1.0547,1.0418,1.0505,287165.37,2021-03-06 03:30:00,0.000381,1.056923,1.060634,...,1.0753,1.0418,25.970149,18.128580,0.0129,0.010057,0.012280,0.009574,0 days 01:45:00,0 days 01:45:00


In [ ]:
strategy = 'i-emas-stats'
strategy_params_json = f'./strategy-params/{strategy}.json'

fee:float = 0.001
# capital = 10000
# risk_per_trade_perc = 0.01
# risk = 0.01
ddf = df
dlen = len(ddf)

sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

strategy_params = {
    # 'mav1': {'type': 'int', 'value': 21, 'step': 1},
    # 'mav2': {'type': 'int', 'value': 50, 'step': 1},
    # 'mav3': {'type': 'int', 'value': 50, 'step': 5},
    # 'mav1mav2': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    # 'mav2mav3': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    # 'stop_gap': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    # 'slipage': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    # 'risk': {'type': 'float', 'value': 0.01, 'step': 0.005},
    # 'sl': {'type': 'float', 'value': 0.01, 'step': 0.001},
    # 'r2r': {'type': 'float', 'value': 2./1., 'step': 0.05},
    # 'wbtp': {'type': 'float', 'value': 0.1, 'step': 0.05},
    # 'trail': {'type': 'float', 'value': 0.02, 'step': 0.005},
}
for pk, pv in strategy_params.items():
    pv['wdg'] = globals()[(pv['wdg'] if 'wdg' in pv else pv['type'].capitalize() + 'Text')](
        description=pk,disabled=False,continuous_update=False,
        value=pv['value'], step=pv['step']
    )
strategy_params = {"w2log": {'wdg': sl_n2}, "w": {'wdg': sl_w}, **strategy_params}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

wst = wed = wsz = False
wdf = wadf = False

def printer(w2log, w):
    # global capital
    capital = 10000
    global wst, wed, wsz, wdf
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))

    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = ddf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    
    plt.close("all")
    vlines=list(wdf[wdf['g_cond_beg_end']].index.astype('string').values)

    with plt.rc_context():
        kwargs = dict(warn_too_much_data=100000)

        wdf = wdf.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})
        fig = plt.figure(constrained_layout=True,figsize=(16,5))
        gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

        gs00 = gs0[0,0].subgridspec(3, 1, height_ratios=[3,1,1])

        ax001 = fig.add_subplot(gs00[0,0])
        ax002 = fig.add_subplot(gs00[1,0], sharex=ax001)
        ax003 = fig.add_subplot(gs00[2,0], sharex=ax001)

        # gs01 = gs0[0,1].subgridspec(4, 1)
        # ax101 = fig.add_subplot(gs01[0,0])
        # ax102 = fig.add_subplot(gs01[1,0])
        # ax103 = fig.add_subplot(gs01[2,0])
        
        locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
        formatter = mdates.ConciseDateFormatter(locator)
        ax001.xaxis.set_major_locator(locator)
        ax001.xaxis.set_major_formatter(formatter)

        mpf.plot(wdf,
            **{**kwargs, **dict(volume=False, style='binance',type='candle')},  #figsize=(15, 5)
            show_nontrading=True,
            vlines=dict(vlines=vlines, linewidths=(1,0.5), colors=('g','c'), linestyle='-.'),
            ax=ax001,
            # vlines=['2021-03-15 04:00:00'],
            # addplot=ap2
            )
        ax001.plot(wdf.ma1, color='orange')
        ax001.plot(wdf.ma2, color='green')
        ax001.plot(wdf.ma3, color='blue')
        
        max_in_lret = wdf[['inlret', 'r_inlret']].max().max()
        min_in_lret = wdf[['inlret', 'r_inlret']].min().min()
        ax002.plot(wdf.r_inlret, lw=1, color='r')
        ax002.plot(wdf.max_r_inlret, lw=1, color='g', linestyle='dashed')
        ax002.plot(wdf.min_r_inlret, lw=1, color='r', linestyle='dashed')
        ax002.vlines(wdf[wdf.g_cond_beg_end].index, min_in_lret, max_in_lret, lw=0.5, color="c")

        ax003.plot(wdf.fair_c_ma2, lw=1, color='g')
       
    plt.show()
    
if exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];
                                        

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()})
display(ui, out)

In [ ]:
df['lo'] = df.o.apply(np.log)
df['lh'] = df.h.apply(np.log)
df['ll'] = df.l.apply(np.log)
df['lc'] = df.c.apply(np.log)

df['ma1'] = df.c.ewm(span=14, adjust=False).mean()
df['ma2'] = df.c.ewm(span=26, adjust=False).mean()
df['ma3'] = df.c.ewm(span=50, adjust=False).mean()

df['lma1'] = df.lc.ewm(span=14, adjust=False).mean()
df['lma2'] = df.lc.ewm(span=26, adjust=False).mean()
df['lma3'] = df.lc.ewm(span=50, adjust=False).mean()


df['fair_c_ma1'] = df.lc - df.lma1
df['fair_c_ma2'] = df.lc - df.lma2
df['fair_c_ma3'] = df.lc - df.lma3

df['fair_ma1_ma2'] = df.lma1 - df.lma2
df['fair_ma2_ma3'] = df.lma2 - df.lma3
df['fair_ma1_ma3'] = df.lma1 - df.lma3

df['lret'] = df.c.apply(np.log).diff()
df = df.tail(-50).copy()

In [ ]:
cond_mav_above = (df.lma2 > df.lma3)
cond_mav_below = (df.lma2 <= df.lma3)
# df['lma23_cross_up'] 
cond_beg = ( cond_mav_above & cond_mav_below.shift(fill_value=False) )
g_beg = cond_beg.cumsum()
# df['lma23_cross_dn']
cond_end = ( (g_beg > 0) & cond_mav_below & cond_mav_above.shift(fill_value=False) )

g_beg = cond_beg.cumsum()
g_end = cond_end.cumsum()
g_inside = (g_beg > g_end)
df['g_beg'] = g_beg
df['g_end'] = g_end
df['g_cond_beg_end'] = (cond_beg | cond_end)
df['g_inside'] = g_inside 
df_inside = df[g_inside].copy()

pos_groups = df_inside.groupby(['g_beg'])
df_inside.loc[pos_groups.head(1).index, 'lret'] = 0.
df_inside['inlret'] = pos_groups['lret'].cumsum(axis=0)
r_pos_groups = df_inside[::-1].groupby(['g_beg'])
df_inside['r_inlret'] = r_pos_groups['lret'].cumsum(axis=0)[::-1]

r_df_inside = df_inside[::-1]
r_pos_groups = r_df_inside.groupby(['g_beg'])
df_inside['max_r_inlret'] = r_pos_groups['r_inlret'].cummax(axis=0)[::-1]
df_inside['min_r_inlret'] = r_pos_groups['r_inlret'].cummin(axis=0)[::-1]
# df_inside['r_inlret'] =  pos_groups['lret'].transform(lambda x: x[::-1].cumsum()[::-1])
# df_inside['r_inlret'] = pos_groups['lret'][::-1].cumsum(axis=0)[::-1]

df.loc[g_inside, 'inlret'] = df_inside['inlret']
df.loc[g_inside, 'r_inlret'] = df_inside['r_inlret']
df.loc[g_inside, 'max_r_inlret'] = df_inside['max_r_inlret']
df.loc[g_inside, 'min_r_inlret'] = df_inside['min_r_inlret']



In [ ]:
df_inside = df[g_inside].copy()
# sns.set(rc={"figure.figsize":(3, 4)})
# plt.figure(figsize=(12, 2))
# sns.set(rc={'figure.figsize':(11.7,8.27)})
# plt.figure(figsize=(15,4))
# sns.displot(ret_r2r, bins=100)
# sns.displot(max_rets, bins=100)
# sns.displot(min_rets, bins=100)
# sns.displot(df_inside, x="fair_c_ma2", y="max_r_inlret", kind="kde")

# sns.displot(df_inside, x="fair_c_ma2", y="max_r_inlret", binwidth=(0.005, .01), cbar=True)
# sns.displot(df_inside, x="fair_ma2_ma3", y="max_r_inlret", binwidth=(0.005, .01), cbar=True)

sns.jointplot(data=df_inside, x="fair_c_ma2", y="max_r_inlret", marker="+")
plt.gcf().set_size_inches(8, 2)
# ret_r2r

In [ ]:

# sns.displot(df_inside, x="fair_c_ma2", y="min_r_inlret", binwidth=(0.005, .01), cbar=True)
sns.displot(df_inside, x="fair_c_ma2", y="min_r_inlret", binwidth=(0.005, .01), cbar=True)
# sns.jointplot(data=df_inside, x="fair_c_ma2", y="min_r_inlret")

# sns.jointplot(data=df_inside, x="fair_c_ma1", y="min_r_inlret", marker="+")
plt.gcf().set_size_inches(8, 2)

In [ ]:
wdf[['inlret', 'r_inlret']].max().max()

In [ ]:
# df['mdate'] = [mdates.date2num(d) for d in df['date']]
wdf['mdate'] = wdf.date.apply(mdates.date2num)
# pd.to_datetime(wdf['date'], unit='s')